In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.init as init

In [2]:
import sys; sys.argv=['']; del sys

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [4]:
class Net_BatchNorm(nn.Module):
    def __init__(self):
        super(Net_BatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.batchN1= nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.batchN2= nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.batchN3 = nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.batchN1(self.conv1(x)))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.batchN2(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.batchN3(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
        

In [5]:
class Net_BatchNorm_DropOut(nn.Module):
    def __init__(self):
        super(Net_BatchNorm_DropOut, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.batchN1= nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.batchN2= nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.batchN3 = nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.batchN1(self.conv1(x)))
        x = self.dropout(x)
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.batchN2(self.conv2(x)))
        x = self.dropout(x)
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.batchN3(self.fc1(x)))
        #x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
class Net_dropOut(nn.Module):
    def __init__(self,dropout=0.5):
        super(Net_dropOut, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = self.dropout(x)
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        #x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [7]:
class Net_sigmoid(nn.Module):
    def __init__(self):
        super(Net_sigmoid, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = torch.sigmoid(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = torch.sigmoid(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [8]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx ==len(train_loader)-1:# args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [9]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
def InitialModel(train_loader,test_loader,device, args):
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

In [11]:
def Model_Sigmoid(train_loader,test_loader,device, args):
    model = Net_sigmoid().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

In [12]:
def Model_DropOut(train_loader,test_loader,device, args):
    for dropout in [0.25,0.5,0.75,1]:
        print("\n\n DropOut is added for dropout value::",dropout,"\n")

        model = Net_dropOut(dropout).to(device)
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

        for epoch in range(1, args.epochs + 1):
            train(args, model, device, train_loader, optimizer, epoch)
            print("Testing set accuracy")
            test(args, model, device, test_loader)
            print("Training set accuracy")
            test(args, model, device, train_loader)

In [13]:
def Model_BatchNorm(train_loader,test_loader,device, args):
    model = Net_BatchNorm().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        print("Testing set accuracy")
        test(args, model, device, test_loader)
        print("Training set accuracy")
        test(args, model, device, train_loader)

In [14]:
def Model_BatchNorm_DropOut(train_loader,test_loader,device, args):
    model = Net_BatchNorm_DropOut().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        print("Testing set accuracy")
        test(args, model, device, test_loader)
        print("Training set accuracy")
        test(args, model, device, train_loader)

In [15]:
def weights_init_Xavier(m):
    if isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        init.normal_(m.bias.data)
        init.xavier_normal_(m.weight.data)

In [16]:
def weights_init_Kaiming(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        init.normal_(m.bias.data)
        init.kaiming_normal_(m.weight.data)

In [17]:
def Model_BatchNorm_InitializeWeights(train_loader,test_loader,device, args,initType):
    model = Net_BatchNorm().to(device)
    if(initType=='Xavier'):
        model.apply(weights_init_Xavier)
    else:
        model.apply(weights_init_Kaiming)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

In [18]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 10000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=1000, metavar='N',
                    help='how many batches to wait before logging training status')


parser.add_argument('--save-model', action='store_true', default=False,
                    help='For Saving the current Model')
args = parser.parse_args()
use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


    

In [19]:
print("For ReLU model:")
InitialModel(train_loader,test_loader,device, args)

For ReLU model:
Train Epoch: 1 [29984/60000 (100%)]	Loss: 0.055160

Test set: Average loss: 0.1020, Accuracy: 9662/10000 (97%)

Train Epoch: 2 [29984/60000 (100%)]	Loss: 0.085563

Test set: Average loss: 0.0613, Accuracy: 9826/10000 (98%)

Train Epoch: 3 [29984/60000 (100%)]	Loss: 0.013297

Test set: Average loss: 0.0564, Accuracy: 9804/10000 (98%)

Train Epoch: 4 [29984/60000 (100%)]	Loss: 0.014588

Test set: Average loss: 0.0409, Accuracy: 9860/10000 (99%)

Train Epoch: 5 [29984/60000 (100%)]	Loss: 0.185066

Test set: Average loss: 0.0382, Accuracy: 9870/10000 (99%)

Train Epoch: 6 [29984/60000 (100%)]	Loss: 0.036793

Test set: Average loss: 0.0333, Accuracy: 9896/10000 (99%)

Train Epoch: 7 [29984/60000 (100%)]	Loss: 0.007713

Test set: Average loss: 0.0345, Accuracy: 9874/10000 (99%)

Train Epoch: 8 [29984/60000 (100%)]	Loss: 0.190038

Test set: Average loss: 0.0390, Accuracy: 9875/10000 (99%)

Train Epoch: 9 [29984/60000 (100%)]	Loss: 0.007439

Test set: Average loss: 0.0288, Accu

In [20]:
print("For Sigmoid model:")
Model_Sigmoid(train_loader,test_loader,device, args)

For Sigmoid model:
Train Epoch: 1 [29984/60000 (100%)]	Loss: 2.313998

Test set: Average loss: 2.3136, Accuracy: 1843/10000 (18%)

Train Epoch: 2 [29984/60000 (100%)]	Loss: 2.299571

Test set: Average loss: 2.2830, Accuracy: 1095/10000 (11%)

Train Epoch: 3 [29984/60000 (100%)]	Loss: 2.081881

Test set: Average loss: 2.0850, Accuracy: 3225/10000 (32%)

Train Epoch: 4 [29984/60000 (100%)]	Loss: 1.076338

Test set: Average loss: 0.9830, Accuracy: 7534/10000 (75%)

Train Epoch: 5 [29984/60000 (100%)]	Loss: 0.670154

Test set: Average loss: 0.5749, Accuracy: 8312/10000 (83%)

Train Epoch: 6 [29984/60000 (100%)]	Loss: 0.470083

Test set: Average loss: 0.4403, Accuracy: 8690/10000 (87%)

Train Epoch: 7 [29984/60000 (100%)]	Loss: 0.434886

Test set: Average loss: 0.3647, Accuracy: 8925/10000 (89%)

Train Epoch: 8 [29984/60000 (100%)]	Loss: 0.324955

Test set: Average loss: 0.3106, Accuracy: 9085/10000 (91%)

Train Epoch: 9 [29984/60000 (100%)]	Loss: 0.457813

Test set: Average loss: 0.2711, A

### Observation:
The testing set accuracy obtained using the RELU model is much better than the Sigmoid Model right from the first epoch.
The sigmoid model supresses the output to a probablity between 0 and 1 ehereas in ReLU, the output still have numerical value >1 if greater than 0.
The major advantage of ReLU over sigmoid is the fact that it is sparse and reduces the likelihood of the vanishing gradient( has values greater than 1 in output).


In [21]:
print("Dropout added to the model::::\n")
args.epochs = 5
Model_DropOut(train_loader,test_loader,device, args)

Dropout added to the model::::



 DropOut is added for dropout value:: 0.25 

Train Epoch: 1 [29984/60000 (100%)]	Loss: 0.062420
Testing set accuracy

Test set: Average loss: 0.1381, Accuracy: 9739/10000 (97%)

Training set accuracy

Test set: Average loss: 0.1492, Accuracy: 58230/60000 (97%)

Train Epoch: 2 [29984/60000 (100%)]	Loss: 0.052462
Testing set accuracy

Test set: Average loss: 0.0767, Accuracy: 9838/10000 (98%)

Training set accuracy

Test set: Average loss: 0.0841, Accuracy: 58909/60000 (98%)

Train Epoch: 3 [29984/60000 (100%)]	Loss: 0.018313
Testing set accuracy

Test set: Average loss: 0.0668, Accuracy: 9853/10000 (99%)

Training set accuracy

Test set: Average loss: 0.0709, Accuracy: 59077/60000 (98%)

Train Epoch: 4 [29984/60000 (100%)]	Loss: 0.142248
Testing set accuracy

Test set: Average loss: 0.0601, Accuracy: 9862/10000 (99%)

Training set accuracy

Test set: Average loss: 0.0605, Accuracy: 59223/60000 (99%)

Train Epoch: 5 [29984/60000 (100%)]	Loss: 0.016588
Te

### Observation:
I applied the dropout layer after both convolution layers and trained it for 5 epochs. As expected for dropout of 1, got very less accuracy for the training and testing set.


Testing error after 5 epochs:
For Dropout 0.25:  Training Testing accuracy : 9881/10000
For Dropout 0.5 :  Testing accuracy : 9885/10000
For Dropout 0.75:  Testing accuracy : 9888/10000
For Dropout 1   :  Testing accuracy : 1133/10000

In [22]:
args.epochs=10
print("BatchNorm, Dropout of 0.5 with RELU")
Model_BatchNorm_DropOut(train_loader,test_loader,device, args)
print("\n \n BatchNorm with RELU")
Model_BatchNorm(train_loader,test_loader,device, args)

BatchNorm, Dropout of 0.5 with RELU
Train Epoch: 1 [29984/60000 (100%)]	Loss: 0.175157
Testing set accuracy

Test set: Average loss: 0.4503, Accuracy: 8886/10000 (89%)

Training set accuracy

Test set: Average loss: 0.4685, Accuracy: 53116/60000 (89%)

Train Epoch: 2 [29984/60000 (100%)]	Loss: 0.046433
Testing set accuracy

Test set: Average loss: 0.3714, Accuracy: 8993/10000 (90%)

Training set accuracy

Test set: Average loss: 0.3894, Accuracy: 53672/60000 (89%)

Train Epoch: 3 [29984/60000 (100%)]	Loss: 0.112126
Testing set accuracy

Test set: Average loss: 0.4470, Accuracy: 8441/10000 (84%)

Training set accuracy

Test set: Average loss: 0.4688, Accuracy: 50007/60000 (83%)

Train Epoch: 4 [29984/60000 (100%)]	Loss: 0.022684
Testing set accuracy

Test set: Average loss: 0.4154, Accuracy: 8555/10000 (86%)

Training set accuracy

Test set: Average loss: 0.4336, Accuracy: 50766/60000 (85%)

Train Epoch: 5 [29984/60000 (100%)]	Loss: 0.031796
Testing set accuracy

Test set: Average loss:

In [23]:
print("BatchNorm,Initialization of weights with Xavier for Conv and Linear with RELU")
Model_BatchNorm_InitializeWeights(train_loader,test_loader,device, args,'Xavier')

BatchNorm,Initialization of weights with Xavier for Conv and Linear with RELU
Train Epoch: 1 [29984/60000 (100%)]	Loss: 0.063291

Test set: Average loss: 0.0566, Accuracy: 9845/10000 (98%)

Train Epoch: 2 [29984/60000 (100%)]	Loss: 0.039832

Test set: Average loss: 0.0459, Accuracy: 9868/10000 (99%)

Train Epoch: 3 [29984/60000 (100%)]	Loss: 0.009261

Test set: Average loss: 0.0349, Accuracy: 9894/10000 (99%)

Train Epoch: 4 [29984/60000 (100%)]	Loss: 0.006500

Test set: Average loss: 0.0335, Accuracy: 9898/10000 (99%)

Train Epoch: 5 [29984/60000 (100%)]	Loss: 0.012372

Test set: Average loss: 0.0290, Accuracy: 9910/10000 (99%)

Train Epoch: 6 [29984/60000 (100%)]	Loss: 0.019899

Test set: Average loss: 0.0261, Accuracy: 9914/10000 (99%)

Train Epoch: 7 [29984/60000 (100%)]	Loss: 0.018776

Test set: Average loss: 0.0277, Accuracy: 9918/10000 (99%)

Train Epoch: 8 [29984/60000 (100%)]	Loss: 0.005522

Test set: Average loss: 0.0252, Accuracy: 9920/10000 (99%)

Train Epoch: 9 [29984/6000

In [24]:
print("BatchNorm,Initialization of weights with Kaiming for Conv and Linear with RELU")
Model_BatchNorm_InitializeWeights(train_loader,test_loader,device, args,'Kaiming')  

BatchNorm,Initialization of weights with Kaiming for Conv and Linear with RELU
Train Epoch: 1 [29984/60000 (100%)]	Loss: 0.052936

Test set: Average loss: 0.0870, Accuracy: 9762/10000 (98%)

Train Epoch: 2 [29984/60000 (100%)]	Loss: 0.033682

Test set: Average loss: 0.0609, Accuracy: 9807/10000 (98%)

Train Epoch: 3 [29984/60000 (100%)]	Loss: 0.007769

Test set: Average loss: 0.0512, Accuracy: 9841/10000 (98%)

Train Epoch: 4 [29984/60000 (100%)]	Loss: 0.030726

Test set: Average loss: 0.0450, Accuracy: 9852/10000 (99%)

Train Epoch: 5 [29984/60000 (100%)]	Loss: 0.187168

Test set: Average loss: 0.0405, Accuracy: 9867/10000 (99%)

Train Epoch: 6 [29984/60000 (100%)]	Loss: 0.003612

Test set: Average loss: 0.0393, Accuracy: 9876/10000 (99%)

Train Epoch: 7 [29984/60000 (100%)]	Loss: 0.060561

Test set: Average loss: 0.0361, Accuracy: 9885/10000 (99%)

Train Epoch: 8 [29984/60000 (100%)]	Loss: 0.010475

Test set: Average loss: 0.0344, Accuracy: 9888/10000 (99%)

Train Epoch: 9 [29984/600